# Free vibrations of a rectangular thin plate

Problem: A thin, homogeneous, isotropic, rectangular plate is clamped on all 4 sides. The plate is set into free vibration by an initial disturbance. Here, we want to study the natural frequencies and the normal modes of the plate. 

_Note_: The natural frequencies and normal modes are the eigenvalues and eigenvectors obtained after solving the transcendental eigenvalue problem for the stiffness and mass matrices.

### Import packages

In [1]:
# Imports
import numpy as np
import ufl
import sys, slepc4py
slepc4py.init(sys.argv)

from dolfinx import fem
from dolfinx.fem import (Constant, dirichletbc, Function, functionspace,
        locate_dofs_topological)
from dolfinx.mesh import (create_box, CellType,
        locate_entities_boundary)
from dolfinx.io import XDMFFile
from dolfinx.fem.petsc import assemble_matrix
from mpi4py import MPI
from slepc4py import SLEPc

### Mesh the thin rectangular plate

In [2]:
# Define computational domain
L = np.array([1, 1.2, 0.02])
Nx=100
N = [Nx, int(L[1]/L[0]*Nx)+1, int(L[2]/L[0]*Nx)+1]
mesh = create_box(MPI.COMM_WORLD, [np.array([0,0,0]), L], N,
        cell_type=CellType.hexahedron)

### Material parameters for the plate

In [3]:
# Material constants
E, nu = (72e9), (0.3)  
rho = (2800)
mu = Constant(mesh, E/2./(1+nu))
lambda_ = Constant(mesh, E*nu/(1+nu)/(1-2*nu))

### Linear elastic constitutive relations

In [4]:
def epsilon(u):
    return ufl.sym(ufl.grad(u))
def sigma(u):
    return 2*mu*epsilon(u) + lambda_ * ufl.operators.tr(epsilon(u)) * ufl.Identity(mesh.geometry.dim)

### Define function space, trial and test functions

In [5]:
V = functionspace(mesh, ("Lagrange", 1,(mesh.geometry.dim,)))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

### Boundary conditions

In [6]:
# define the boundaries
def left_boundary(x):
    return np.isclose(x[0],0)
def right_boundary(x):
    return np.isclose(x[0],L[0])
def bot_boundary(x):
    return np.isclose(x[1],0)
def top_boundary(x):
    return np.isclose(x[1],L[1])

# find the facets on the boundaries
fdim = mesh.topology.dim - 1
left_facets = locate_entities_boundary(mesh, fdim, left_boundary)
right_facets = locate_entities_boundary(mesh, fdim, right_boundary)
bot_facets = locate_entities_boundary(mesh, fdim, bot_boundary)
top_facets = locate_entities_boundary(mesh, fdim, top_boundary)

# apply fixed (clamped) Dirichlet boundary conditions to all facets
u_D = Function(V)
u_D.interpolate(lambda x: np.zeros_like(x))
left_bc = dirichletbc(u_D, locate_dofs_topological(V, fdim, left_facets))
right_bc = dirichletbc(u_D, locate_dofs_topological(V, fdim, right_facets))
bot_bc = dirichletbc(u_D, locate_dofs_topological(V, fdim, bot_facets))
top_bc = dirichletbc(u_D, locate_dofs_topological(V, fdim, top_facets))
bcs=[left_bc,right_bc,bot_bc,top_bc]

### Variational form of the stiffness and mass matrices

In [7]:
k_form = ufl.inner(sigma(v),epsilon(u))*ufl.dx
m_form = rho*ufl.inner(u,v)*ufl.dx

### Assemble the stiffness and mass matrices

In [ ]:
# Using the "diagonal" kwarg ensures that Dirichlet BC modes will not be among
# the lowest-frequency modes of the beam. 
K = assemble_matrix(fem.form(k_form), bcs=bcs, diagonal=62831)
M = assemble_matrix(fem.form(m_form), bcs=bcs, diagonal=1/62831)
K.assemble()
M.assemble()

### Create and configure the eigenvalue solver

In [11]:
N_eig = 6 # number of eigenvalues to compute
eigensolver = SLEPc.EPS().create(MPI.COMM_WORLD) # initialize eigen problem solver
eigensolver.setDimensions(N_eig) # sets the number of eigenvalues to compute
eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP) # set problem type as the Generalized Hermitian Eigenvalue Problem
st = SLEPc.ST().create(MPI.COMM_WORLD) # create a spectral transformation object to improve convergence
st.setType(SLEPc.ST.Type.SINVERT) # spectral transformation type: shift and invert
st.setShift(0.1) # set the shift value for the spectral transformation
st.setFromOptions() # allow user to pass additional settings via command line
eigensolver.setST(st)
eigensolver.setOperators(K, M)
eigensolver.setFromOptions()

### Compute the eigenvalue-eigenvector pairs

In [12]:
eigensolver.solve()
evs = eigensolver.getConverged()
vr, vi = K.getVecs()
u_output = Function(V)
u_output.name = "Eigenvector"
print( "Number of converged eigenpairs %d" % evs )
if evs > 0:
    with XDMFFile(MPI.COMM_WORLD, f"eigenvectors_Nx{int(Nx)}.xdmf", "w") as xdmf:
        xdmf.write_mesh(mesh)
        for i in range (min(N_eig, evs)):
            l = eigensolver.getEigenpair(i, vr, vi)
            freq = np.sqrt(l.real)/2/np.pi
            print(f"Mode {i}: {freq} Hz")
            u_output.x.array[:] = vr
            xdmf.write_function(u_output, i)

Number of converged eigenpairs 6
Mode 0: 156.8248030848762 Hz
Mode 1: 282.5499553187332 Hz
Mode 2: 352.7002090459315 Hz
Mode 3: 464.98129596969886 Hz
Mode 4: 485.9148879625408 Hz
Mode 5: 651.79825124868 Hz


### Convert the natural frequencies to the dimensionless frequencies

In [14]:
freq_arr=np.array([156.8248030848762,282.5499553187332,352.700290459315,464.98129596969886,485.9148879625408,651.79825124868])
D=E*L[2]**3 / (12*(1-nu**2))
dimless_freq_arr=np.power( rho*L[2]*((freq_arr*2*np.pi)**2)/D , 1/4 )

### Compare FEA dimensionless frequencies to exact solution

In [ ]:
exact_arr = np.array([5.49,7.42,8.30,9.56,9.76,11.31])
print(f'FEA dimensionless frequencies={np.round(dimless_freq_arr,2)}')
print(f'exact dimensionless frequencies={exact_arr}')

FEA dimensionless frequencies=[ 5.67  7.61  8.5   9.76  9.97 11.55]
exact dimensionless frequencies=[ 5.49  7.42  8.3   9.56  9.76 11.31]


There are 2 main sources of errors:
- Mesh too coarse: If we used a more refined mesh, we would get a closer result. For running this tutorial, we purposefully used a coarse mesh to demonstrate the code working.
- Different element types: In the paper, the author uses the "Bending Panel element" from MSC Nastran for the FEA portion. Here, we're using a hexahedron element.